In [1]:
""" 
Historical Data For the Scanner: 
    - This module gets historical prices for contracts previously identified from the scanner. 
    - It will be used to track the change in contract prices 
    - This will help us determine if our strategy is working or not. 
    - By default, we will find the price of each contract on the day of expiration
        : We are also interested in knowing if there was a maximum profit opportunity, where the observed price was higher than the starting price. 
"""

import pandas as pd 
import numpy as np 
import sqlite3 as sql
import datetime as dt
import sys
sys.path.append('/Users/jerald/Documents/Dir/Python/stocker')
from bin.options.optgd.db_connect import Connector
from bin.main import get_path


connections = get_path('../../../../')

In [2]:
class Tracker(Connector):
    def __init__(self, connections):
        """
        The tracking class, will be used to track option prices for contracts that we have identified. 

        
        Methdos:
            - Utils:
                1. __load_tracking_(stock): Load tracking table
                2. __create_tracking_table(stock): Create a tracking table for the stock
                3. __update_tracking_table(stock, data): Update the tracking table for the stock
                4. __get_contract_info(stock, contractsymbol): Get the contract information including price for the given stock and symbol
                5. __estimate_current_price(df): Estimate the price of the contract from the given dataframe containing the contract information
                
            - Tracking Methods: 
                1. track(stock, contractsymbol, date): 
                    Track the price of the contract from the date
                2. calculate_pl(stock, contractsymbol): 
                    Calculate the profit or loss for the contract
                3. calculate_max_profit(stock, contractsymbol):
                    Calculate the maximum profit for the contract
                4. calculate_max_loss(stock, contractsymbol):
                    Calculate the maximum loss for the contract

        Intialize With: 
        Args:
            connections (_type_): _description_
        """
        
        super().__init__(connections)
        self.verbose = True
        self.conn = self.tracking_db
        
    def __load_tracking_table(self, stock):
        """
        Load the tracking table for the stock
        """
        query = f"SELECT * FROM {stock}"
        return pd.read_sql(query, self.conn)
    
    def __create_tracking_table(self, stock):
        """
        Create a tracking table for the stock
        """
        query = f"CREATE TABLE {stock} (contract TEXT, date TEXT, price REAL)"
        self.conn.execute(query)
        self.conn.commit()
    
    def __update_tracking_table(self, stock, data):
        """
        Update the tracking table for the stock
        """
        query = f"INSERT INTO {stock} (contract, date, price) VALUES (?, ?, ?)"
        self.conn.executemany(query, data)
        self.conn.commit()
    
    def __get_contract_info(self, stock, contractsymbol, date = None):
        """
        Get the contract information including price for the given stock and symbol
        """
        if date == None: 
            query = f"SELECT * FROM {stock} WHERE contractsymbol = '{contractsymbol}'"
        else:
            query = f"""SELECT * FROM {stock} WHERE contractsymbol = '{contractsymbol}' 
                    AND date(gatherdate) >= date('{date}')"""
        return pd.read_sql(query, self.change_db, parse_dates = ['gatherdate'])
    
    def track(self, stock, contractsymbol, date = None):
        """
        Track the price of the contract from the date
        """
        # Get the contract information
        contract_info = self.__get_contract_info(stock, contractsymbol, date)
        
        # Get the price of the contract on the given date
        return contract_info
    
    def calculate_pl(self, contract_info):
        """
        Calculate the profit given the dataframe containing the contract information
        The columns we will use here is: 
            - gatherdate: The date the price was gathered
            - lastprice: The price of the contract
            - lasprice_avg_5d: The average price of the contract over the last 5 days
            - lastprice_avg_30d: The average price of the contract over the last 30 days
            - impliedvolatility: The implied volatility of the contract
            - iv_avg_5d: The average implied volatility of the contract over the last 5 days
            - iv_avg_30d: The average implied volatility of the contract over the last 30 days
            
        Create the following columns based on purchasing N contracts: 
            - cost: The cost of purchasing N contracts
            - current_value: The current value of the contract
            - pl: The profit or loss of the contract
            - pl_percent: The profit or loss percentage of the contract
        """
        N = 100
        cols_like = contract_info.filter(regex = 'lastprice|impliedvolatility|iv').columns
        cols_like = ['gatherdate', 'contractsymbol'] + cols_like.tolist()
        contract_info = contract_info[cols_like].copy()
        contract_info['cost'] = N * contract_info['lastprice'].iloc[0] * 100 
        contract_info['value'] = N * contract_info['lastprice'] * 100 
        contract_info['pl'] = contract_info['value'] - contract_info['cost']
        contract_info['pl_percent'] = (contract_info['pl'] / contract_info['cost']) * 100
        return contract_info
        
    

# Test        
t = Tracker(connections)

Options db Connected: 2024-09-30 18:57:55.557639


In [3]:
cs = 'ADBE240927C00542500'
date = '2024-09-20'

d = t.track('ADBE', cs, date)
t.calculate_pl(d)

,gatherdate,contractsymbol,lastprice,impliedvolatility,lastprice_chg,lastprice_avg_30d,lastprice_avg_5d,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,cost,value,pl,pl_percent
0,2024-09-20 15:28:18,ADBE240927C00542500,0.88,0.224739,-1.06,2.278000,2.278000,-0.010254,0.284223,0.284223,0.284223,8800.0,8800.0,0.0,0.000000
1,2024-09-23 15:16:53,ADBE240927C00542500,1.37,0.238533,0.49,2.126667,2.126667,0.013794,0.276608,0.276608,0.276608,8800.0,13700.0,4900.0,55.681818
2,2024-09-24 13:55:41,ADBE240927C00542500,0.58,0.263557,-0.79,1.905714,1.406667,0.025024,0.276090,0.274744,0.274744,8800.0,5800.0,-3000.0,-34.090909
3,2024-09-25 13:57:49,ADBE240927C00542500,0.14,0.294441,-0.44,1.685000,1.011667,0.030883,0.276842,0.277206,0.277206,8800.0,1400.0,-7400.0,-84.090909
4,2024-09-26 15:16:58,ADBE240927C00542500,0.03,0.370123,-0.11,1.501111,0.823333,0.075683,0.271064,0.287530,0.287530,8800.0,300.0,-8500.0,-96.590909
5,2024-09-27 15:27:08,ADBE240927C00542500,0.01,0.367194,-0.02,1.352000,0.501667,-0.002930,0.293098,0.295497,0.295497,8800.0,100.0,-8700.0,-98.863636


In [7]:
contracts = pd.read_sql('select * from amnt where amnt > 1000', t.stats_db)

contracts

,gatherdate,contractsymbol,stk_price,lastprice,ask,bid,change,percentchange,volume,openinterest,...,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,vol_chg,oi_chg,flag,amnt,voi,iv_diff
0,2024-09-30 14:31:42,IWM241115P00209000,218.98,3.20,3.20,3.16,-0.11,-3.323260,15,48406,...,-0.011658,0.236020,0.227759,0.181772,4.0,46150.0,1,46139,0.0,-0.003999
1,2024-09-30 14:05:50,SPY241115C00580000,571.38,9.54,9.59,9.55,-0.43,-4.312942,1295,45309,...,0.001236,0.157482,0.147433,0.144788,451.0,40354.0,1,39510,0.0,-0.015936
2,2024-09-30 14:25:22,TLT241004C00098500,98.02,0.49,0.50,0.48,-0.07,-12.499998,4556,24969,...,0.046386,0.124358,0.146654,0.146654,1862.0,22652.0,1,19958,0.0,-0.009116
3,2024-09-30 14:27:55,NVDA241004P00050000,119.35,0.01,0.01,0.00,0.00,0.000000,102,22386,...,0.500003,1.822919,1.375238,1.375238,100.0,17627.0,1,17625,0.0,-0.999766
4,2024-09-30 14:05:50,SPY241025P00555000,571.38,3.15,3.20,3.18,-0.32,-9.221900,177,18914,...,0.009521,0.140450,0.141148,0.141148,-158.0,17399.0,1,17064,0.0,-0.011479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2024-09-30 14:11:52,VALE241220C00013000,11.79,0.32,0.31,0.30,0.04,14.285711,1976,27447,...,0.004883,0.317715,0.327839,0.327956,1530.0,1604.0,1,1158,0.0,-0.000293
61,2024-09-30 14:05:22,AMZN241018P00140000,186.30,0.03,0.04,0.03,-0.01,-25.000000,5,6679,...,0.021484,0.489263,0.416119,0.349398,-75.0,1191.0,1,1111,0.0,-0.089745
62,2024-09-30 14:09:27,MMM241004C00141000,136.19,0.12,0.12,0.09,-0.17,-58.620686,42,1349,...,0.028320,0.187020,0.227974,0.227974,-72.0,1221.0,1,1107,0.0,0.018982
63,2024-09-30 14:30:32,SNOW250221C00170000,114.04,2.24,2.25,2.15,0.10,4.672893,280,8931,...,0.014404,0.481288,0.476342,0.501019,77.0,1295.0,1,1092,0.0,-0.028424
